Dataset Loading and Preprocessing:

The program loads a customer churn dataset from Google Drive.
Preprocessing includes separating the features (input) and target (output) variables and identifying categorical and numerical columns.
Categorical variables are one-hot encoded, and numerical features are standardized using StandardScaler.
The dataset is split into training, validation, and test sets for model evaluation.
Model Definition:

A function build_model(hp) is defined to create a neural network using two hidden layers.
Hyperparameters such as the number of units in each layer, activation function, and learning rate are tuned dynamically through Keras Tuner.
The model is compiled using the Adam optimizer with a binary cross-entropy loss function and accuracy as a performance metric.
Hyperparameter Tuning:

The program leverages different hyperparameter optimization techniques, such as Random Search, Bayesian Optimization, and Hyperband, to explore the hyperparameter space and find the best model.
The model is evaluated on a test set after tuning to assess its performance.
Performance Evaluation:

The evaluate_model function is used to assess the model on the test set by calculating the test loss.
Results for different optimization techniques, such as test loss and time taken, are collected and displayed in a table for comparison.

In [1]:
# Install keras-tuner
!pip install keras-tuner -q

# Import necessary libraries
import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch, BayesianOptimization, Hyperband
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load dataset
file_path = 'Churn_Modelling.csv'
data = pd.read_csv(file_path)

# Preprocessing: Separate features and target variable
X = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
y = data['Exited']

# Identify categorical and numerical features
categorical_features = ['Geography', 'Gender']
numerical_features = X.drop(columns=categorical_features).columns.tolist()

# Use ColumnTransformer to apply different preprocessing steps to categorical and numerical columns
data_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Split into training (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Split temp data into 50% validation, 50% test (which results in 20% validation, 20% test of full data)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Apply preprocessing (only fit on training data)
X_train_scaled = data_transformer.fit_transform(X_train)  # Fit only on training data
X_val_scaled = data_transformer.transform(X_val)          # Transform validation data
X_test_scaled = data_transformer.transform(X_test)        # Transform test data



# Define the model-building function for tuning with two hidden layers and added 'sigmoid' as an activation choice
#The function must be defined to dynamically create the neural network model with different hyperparameters.
#The function takes hp (hyperparameter object) as input and builds the model based on the values provided during each trial.
#The function returns a compiled model, which Keras Tuner uses for training and evaluation.
def build_model(hp):
    model = Sequential()

    # First hidden layer
    model.add(Dense(units=hp.Int('units_0', min_value=32, max_value=128, step=32),
                    activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid']),
                    input_shape=(X_train_scaled.shape[1],)))

    # Second hidden layer
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32),
                    activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Function to evaluate model on the test set and return loss
def evaluate_model(model, X_test, y_test):
    test_loss = model.evaluate(X_test, y_test, verbose=0)[0]  # Return only test loss
    return test_loss

# Set number of trials (consistent across all methods)
max_trials = 10
max_epochs = 3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.5 MB/s eta 0:00:00


In [2]:
# Random Search with overwrite=True to force re-run
random_tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=max_trials,
    directory='random_search',
    project_name='customer_churn',
    overwrite=True
)

start_time = time.time()
#The search() method of Keras Tuner starts the hyperparameter tuning process, where it tries different combinations of hyperparameters
#and trains the model with those values.
random_tuner.search(X_train_scaled, y_train, epochs=max_epochs, validation_data=(X_val_scaled, y_val), verbose=0)
random_search_time = time.time() - start_time

# Get best model and evaluate on the test set
random_search_best_model = random_tuner.get_best_models(num_models=1)[0]
random_search_test_loss = evaluate_model(random_search_best_model, X_test_scaled, y_test)
random_search_best_hyperparameters = random_tuner.get_best_hyperparameters(num_trials=1)[0].values

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
# Bayesian Optimization with overwrite=True
bayesian_tuner = BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=max_trials,
    directory='bayesian_opt',
    project_name='customer_churn',
    overwrite=True
)

start_time = time.time()
bayesian_tuner.search(X_train_scaled, y_train, epochs=max_epochs, validation_data=(X_val_scaled, y_val), verbose=0)
bayesian_search_time = time.time() - start_time

# Get best model and evaluate on the test set
bayesian_search_best_model = bayesian_tuner.get_best_models(num_models=1)[0]
bayesian_search_test_loss = evaluate_model(bayesian_search_best_model, X_test_scaled, y_test)
bayesian_search_best_hyperparameters = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0].values

In [4]:
# Hyperband with overwrite=True
hyperband_tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=max_epochs,
    directory='hyperband',
    project_name='customer_churn',
    overwrite=True
)

start_time = time.time()
hyperband_tuner.search(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), verbose=0)
hyperband_search_time = time.time() - start_time

# Get best model and evaluate on the test set
hyperband_search_best_model = hyperband_tuner.get_best_models(num_models=1)[0]
hyperband_search_test_loss = evaluate_model(hyperband_search_best_model, X_test_scaled, y_test)
hyperband_search_best_hyperparameters = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0].values

In [5]:
# Adjust Pandas setting to display full DataFrame content
pd.set_option('display.max_colwidth', None)

# Collecting the results
results = {
    "Method": ["Random Search", "Bayesian Optimization", "Hyperband"],
    "Test Loss": [random_search_test_loss, bayesian_search_test_loss, hyperband_search_test_loss],
    "Time (seconds)": [random_search_time, bayesian_search_time, hyperband_search_time],
    "Best Hyperparameters": [
        random_search_best_hyperparameters,
        bayesian_search_best_hyperparameters,
        hyperband_search_best_hyperparameters
    ]
}

# Creating a DataFrame to display the results
results_df = pd.DataFrame(results)
print(results_df)

                  Method     Test Loss  Time (seconds)  \
0          Random Search  2.306187e-08       40.533493   
1  Bayesian Optimization  6.437411e-07       38.662306   
2              Hyperband  9.167569e-10       19.769247   

                                                                                                                                                                       Best Hyperparameters  
0                                                                                                             {'units_0': 128, 'activation': 'tanh', 'units_1': 128, 'learning_rate': 0.01}  
1                                                                                                              {'units_0': 128, 'activation': 'tanh', 'units_1': 96, 'learning_rate': 0.01}  
2  {'units_0': 96, 'activation': 'relu', 'units_1': 32, 'learning_rate': 0.01, 'tuner/epochs': 3, 'tuner/initial_epoch': 1, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0001'}  


In [6]:
bayesian_tuner.results_summary()



Results summary
Results in bayesian_opt/customer_churn
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 00 summary
Hyperparameters:
units_0: 128
activation: tanh
units_1: 96
learning_rate: 0.01
Score: 4.213825661736337e-07

Trial 06 summary
Hyperparameters:
units_0: 64
activation: tanh
units_1: 64
learning_rate: 0.01
Score: 6.465636943175923e-06

Trial 07 summary
Hyperparameters:
units_0: 96
activation: relu
units_1: 128
learning_rate: 0.001
Score: 0.00010782071331050247

Trial 01 summary
Hyperparameters:
units_0: 64
activation: sigmoid
units_1: 96
learning_rate: 0.01
Score: 0.00015658655320294201

Trial 04 summary
Hyperparameters:
units_0: 64
activation: relu
units_1: 96
learning_rate: 0.001
Score: 0.00015830383927095681

Trial 09 summary
Hyperparameters:
units_0: 96
activation: relu
units_1: 64
learning_rate: 0.001
Score: 0.00029572879429906607

Trial 02 summary
Hyperparameters:
units_0: 96
activation: tanh
units_1: 128
learning_rate: 0.0001
Score: 0.02115425

In [7]:
hyperband_tuner.results_summary()

Results summary
Results in hyperband/customer_churn
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0003 summary
Hyperparameters:
units_0: 96
activation: relu
units_1: 32
learning_rate: 0.01
tuner/epochs: 3
tuner/initial_epoch: 1
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0001
Score: 1.1347520700866198e-09

Trial 0001 summary
Hyperparameters:
units_0: 96
activation: relu
units_1: 32
learning_rate: 0.01
tuner/epochs: 1
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 8.721873200556729e-06

Trial 0000 summary
Hyperparameters:
units_0: 64
activation: relu
units_1: 32
learning_rate: 0.01
tuner/epochs: 1
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 4.3547366658458486e-05

Trial 0002 summary
Hyperparameters:
units_0: 32
activation: relu
units_1: 64
learning_rate: 0.01
tuner/epochs: 1
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 5.019347372581251e-05

Trial 0005 summary
Hyperparameters:
units_0: 128
activation: relu